In [1]:
# Load libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from time import time
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
import warnings
import math
from sklearn.cluster import KMeans
import pickle
warnings.filterwarnings('ignore')
%matplotlib inline

### Load .csv files

In [2]:
# load default data frames
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
artists_df = pd.read_csv("artists.csv")
global_median_df = pd.read_csv("global_median.csv")
profiles = pd.read_csv("profiles.csv")

### Create profiles df, imput ages

In [3]:
# CREATES PROFILES DF

# # condition = (profiles_df['age'].isnull()) | (profiles_df['age'] < 95) | (profiles_df['age'] > 0)
# # profiles_df[condition] = np.nan
# profiles_df.loc[profiles_df["age"] < 5 ,'age'] = None
# profiles_df.loc[profiles_df["age"] > 85 ,'age'] = None
# profiles_df.loc[profiles_df["age"].isnull(),'age'] = None

# # calculate mean age per user
# mean_age = profiles_df[~profiles_df['age'].isnull()]["age"].mean()

# ###################################################################### 

# # calculate mean age per men
# males_only_df = profiles_df[profiles_df["sex"]=='m']
# mean_male_age = males_only_df[~males_only_df['sex'].isnull()]["age"].mean()

# ###################################################################### 

# # calculate mean age per women
# females_only_df = profiles_df[profiles_df["sex"]=='f']
# mean_female_age = females_only_df[~females_only_df['sex'].isnull()]["age"].mean()

# ###################################################################### 

# # get list of unique countries
# countries = profiles_df["country"].unique()

# # calculate mean age per country
# mean_country_age = {}
# for country in countries:
#     tmp_df = profiles_df[profiles_df["country"]==str(country)]
#     mean_country_age[str(country)] = tmp_df["age"].mean()

# ###################################################################### 
    
# # calculate mean age per male per country
# mean_country_male_age = {}
# for country in countries:
#     tmp_df0 = profiles_df[profiles_df["sex"]=='m']
#     tmp_df1 = tmp_df0[profiles_df["country"]==str(country)]
#     mean_country_male_age[str(country)] = tmp_df1["age"].mean()

# ###################################################################### 

# # calculate mean age per female per country
# mean_country_female_age = {}
# for country in countries:
#     tmp_df0 = profiles_df[profiles_df["sex"]=='f']
#     tmp_df1 = tmp_df0[profiles_df["country"]==str(country)]
#     mean_country_female_age[str(country)] = tmp_df1["age"].mean()

# ###################################################################### 

# # initialize training df
# profiles = profiles_df

# ###################################################################### 

# # create sex indicators
# profiles["male"] = 0
# profiles["female"] = 0
# profiles["sex_missing"] = 0
# profiles.loc[profiles["sex"] =='m', "male"] = 1
# profiles.loc[profiles["sex"] =='f', "female"] = 1
# profiles.loc[profiles["sex"].isnull(), "sex_missing"] = 1

# ######################################################################  

# # create age indicators
# profiles["age_missing"] = 0
# profiles.loc[profiles["age"].isnull(),"age_missing"] = 1

# # fill in mean age: has no gender or country
# profiles.loc[profiles["age_missing"] == 1, "age"] = mean_age

# # fill in mean age: has gender, no country
# #males
# condition = (profiles["sex"] =='m') & (profiles["country"].isnull()) & (profiles["age_missing"] == 1)
# profiles.loc[condition, "age"] = mean_male_age
# # females
# condition = (profiles["sex"] =='f') & (profiles["country"].isnull()) & (profiles["age_missing"] == 1)
# profiles.loc[condition, "age"] = mean_female_age

# # fill in mean age: has country, no gender
# #loop over countries
# for country in countries:
#     condition = (profiles["country"] == country) & (profiles["age_missing"] == 1) & (profiles["sex_missing"] == 1)
#     profiles.loc[condition, "age"] = mean_country_age[country]

# # fill in mean age: has gender and country
# # loop over countries
# for country in countries:
#     # males
#     condition = (profiles["country"] == country) & (profiles["sex"] == 'm') & (profiles["age_missing"] == 1)
#     profiles.loc[condition, "age"] = mean_country_male_age[country]
#     # females
#     condition = (profiles["country"] == country) & (profiles["sex"] == 'f') & (profiles["age_missing"] == 1)
#     profiles.loc[condition, "age"] = mean_country_female_age[country]

# ###################################################################### 
    
# # create country indicators
# profiles["country_missing"] = 0
# profiles.loc[profiles["country"].isnull(), "country_missing"] = 1
# # loop over countries
# for country in countries:
#     # assign 1 if in that country
#     profiles.loc[profiles["country"] == country, country] = 1
#     # assign 0 otherwise
#     profiles.loc[profiles["country"] != country, country] = 0

# ###################################################################### 
    
# # create country indicators
# profiles["country_missing"] = 0
# profiles.loc[profiles["country"].isnull(), "country_missing"] = 1
# # loop over countries
# for country in countries:
#     # assign 1 if in that country
#     profiles.loc[profiles["country"] == country, country] = 1
#     # assign 0 otherwise
#     profiles.loc[profiles["country"] != country, country] = 0

# profiles.to_csv("profiles.csv")

# ###################################################################### 

# clean profiles df
profiles.drop("sex",inplace=True,axis=1)
profiles.drop("country",inplace=True,axis=1)
profiles.drop("Unnamed: 0",axis=1,inplace=True)
profiles.set_index("user",inplace=True)
profiles.drop("sex_missing",inplace=True,axis=1)
profiles.head(1)

,age,male,female,age_missing,country_missing,Sweden,Iceland,United States,Germany,Netherlands,...,Myanmar,Somalia,Gambia,Mozambique,Rwanda,Saint Kitts and Nevis,New Caledonia,Sudan,Guyana,Marshall Islands
user,,,,,,,,,,,,,,,,,,,,,
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Grab genres for each artist

In [4]:
# load genre data frames and do some cleaning
artists_with_1genre_df = pd.read_csv("artists_with_genres.csv")
artists_with_5genres_df = pd.read_csv("artists_with_top_5_genres.csv")
artists_with_5genres_df.rename(columns = {"num_votes5":"extra"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre5":"num_votes5"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes4":"genre5"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre4":"num_votes4"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes3":"genre4"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre3":"num_votes3"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes2":"genre3"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre2":"num_votes2"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes1":"genre2"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre1":"num_votes1"}, inplace = True)
artists_with_5genres_df.rename(columns = {"name":"genre1"}, inplace = True)
artists_with_5genres_df.rename(columns = {"artist":"name"}, inplace = True)

# Calculate percent of votes
agn = artists_with_5genres_df[["num_votes1","num_votes2","num_votes3"]]
agn.rename(columns = {"num_votes1":"per_votes1"}, inplace = True)
agn.rename(columns = {"num_votes2":"per_votes2"}, inplace = True)
agn.rename(columns = {"num_votes3":"per_votes3"}, inplace = True)
agn = agn.apply(lambda c: c / c.sum() * 100, axis=1)

# combine dfs together
artists_with_5genres_df = pd.concat([artists_with_5genres_df, agn], axis=1)
artists_with_5genres_df.head(1)

,name,genre1,num_votes1,genre2,num_votes2,genre3,num_votes3,genre4,num_votes4,genre5,num_votes5,extra,per_votes1,per_votes2,per_votes3
03098741-08b3-4dd7-b3f6-1b0bfa2c879c,Liars,alternative rock,1,art punk,1,dance-punk,1,experimental rock,1,indie rock,1,NaN,33.333333,33.333333,33.333333


### Create train df

In [5]:
# Train - Test Split on profiles to ensure users are in one group or another
train, test = train_test_split(train_df, test_size = .1)
# train, valid = train_test_split(train0, test_size = .1)
#train = train_df.copy()

In [6]:
# def genre_select(x,at5g):
#     artist = x[0]
#     genres = np.array([at5g.loc[artist,"genre1"],at5g.loc[artist,"genre2"],at5g.loc[artist,"genre3"]])
#     votes = np.array([at5g.loc[artist,"per_votes1"],at5g.loc[artist,"per_votes2"],at5g.loc[artist,"per_votes3"]])
#     votes_tot = np.sum(votes)
#     if votes_tot != 0:
#         weights = votes/float(votes_tot)
#         return np.random.choice(genres, p= weights)
#     else:
#         return None

In [7]:
#add primary genre to train
train["genre1"] = np.nan
train.set_index("artist", inplace=True)
train.update(artists_with_5genres_df)
train.reset_index(inplace=True)

test["genre1"] = np.nan
test.set_index("artist", inplace=True)
test.update(artists_with_5genres_df)
test.reset_index(inplace=True)

In [8]:
# train.genre = train.apply(lambda x: genre_select(x,artists_with_5genres_df),axis = 1)

In [9]:
# train.to_csv("train_wGenreChoice.csv")

### compute averages

In [10]:
# calculate global median
global_median = global_median_df.iloc[0,1]

In [ ]:
# calculate user medians
user_medians = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').median(axis=1)
user_medians = pd.DataFrame(user_medians)
user_medians.rename(columns={0:"user median"},inplace=True)
user_medians.head(1)

,user median
user,
00000c289a1829a808ac09c00daf10bc3c4e223b,231


In [ ]:
# calculate user means
user_means = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').mean(axis=1)
user_means = pd.DataFrame(user_means)
user_means.rename(columns={0:"user mean"},inplace=True)
user_means.head(1)

In [ ]:
# calculate artist medians
artist_medians = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').median(axis=0)
artist_medians = pd.DataFrame(artist_medians)
artist_medians.rename(columns={0:"artist median"},inplace=True)
artist_medians.head(1)

In [ ]:
# calculate artist medians
artist_means = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').mean(axis=0)
artist_means = pd.DataFrame(artist_means)
artist_means.rename(columns={0:"artist mean"},inplace=True)
artist_means.head(1)

In [ ]:
# calculate genre medians
genre_medians = train.groupby("genre1").plays.median()
genre_medians = pd.DataFrame(genre_medians)
genre_medians.rename(columns={'plays':"genre median"},inplace=True)
genre_medians.head(1)

In [ ]:
# calculate genre mean
genre_means = train.groupby("genre1").plays.mean()
genre_means = pd.DataFrame(genre_means)
genre_means.rename(columns={'plays':"genre mean"},inplace=True)
genre_means.head(1)

In [ ]:
# calculate genre_user medians
user_genre_medians = train.groupby(["user","genre1"]).median().reset_index()
user_genre_medians.rename(columns={"plays": "user genre median"},inplace=True)
user_genre_medians.head(1)

In [ ]:
# calculate genre_user means
user_genre_means = train.groupby(["user","genre1"]).mean().reset_index()
user_genre_means.rename(columns={"plays": "user genre mean"},inplace=True)
user_genre_means.head(1)

In [ ]:
#pivot training data to have one row per user
train = train.pivot(index='user',columns='artist',values='plays').reset_index()
#train = train.fillna(0)

In [ ]:
# join profiles and train together
train.reset_index(inplace=True)
train.set_index('user',inplace = True)
train.drop("index",axis=1,inplace=True)

profiles.reset_index(inplace=True)
profiles.set_index('user', inplace=True)

bigdf = pd.concat([profiles,train],axis=1)
bigdf.fillna(0,inplace=True)
bigdf.head(1)

In [ ]:
scaler = StandardScaler()
bigdf_sc = scaler.fit_transform(bigdf)

In [ ]:
kmeans = KMeans(init = 'k-means++', n_clusters=50, n_jobs = -1)
kmeans.fit(bigdf_sc)

In [ ]:
from sklearn.externals import joblib
joblib.dump(kmeans, 'filename.pkl') 

In [ ]:
cols = bigdf.columns
centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_))
centers.columns = cols
centers.head()

### Create test array to compare to kmeans

In [ ]:
closest_centers = kmeans.predict(bigdf_sc)
train['closest center'] = closest_centers.tolist()
test['closest center'] = None
test.set_index("user",inplace=True)
test.update(train)
test.reset_index(inplace=True)

In [ ]:
def get_prediction(x):
    artist = x[1]
    return centers.ix[x[5],artist]

In [ ]:
test["kmeans pred"] = 0
test["kmeans pred"] = test.apply(lambda x: get_prediction(x),axis=1)
test.head()

In [ ]:
# initialize columns
test['user genre median'] = np.nan
test['user median'] = np.nan
test['artist median'] = np.nan
test['genre median'] = np.nan
test['global median'] = np.nan
test['user mean'] = np.nan
test['genre mean'] = np.nan
test['artist mean'] = np.nan
test['user genre mean'] = np.nan

# add global median
test['global median'] = global_median

# add user median
test.set_index('user',inplace=True)
test.update(user_medians)
test.reset_index(inplace=True)

# add artist median
test.set_index('artist',inplace=True)
test.update(artist_medians)
test.reset_index(inplace=True)
test["genre median"].fillna(test["user median"],inplace=True)

# add genre median
test.set_index('genre1',inplace=True)
test.update(genre_medians)
test.reset_index(inplace=True)
test["genre median"].fillna(test["user median"],inplace=True)

# add user genre median
test.set_index(['user','genre1'],inplace=True)
test.update(user_genre_medians.set_index(["user","genre1"]))
test.reset_index(inplace=True)
test["user genre median"].fillna(test["user median"],inplace=True)

# add user mean
test.set_index('user',inplace=True)
test.update(user_means)
test.reset_index(inplace=True)

# add artist mean
test.set_index('artist',inplace=True)
test.update(artist_means)
test.reset_index(inplace=True)
test["genre mean"].fillna(test["user mean"],inplace=True)

# add genre mean
test.set_index('genre1',inplace=True)
test.update(genre_means)
test.reset_index(inplace=True)
test["genre mean"].fillna(test["user mean"],inplace=True)

# add user genre mean
test.set_index(['user','genre1'],inplace=True)
test.update(user_genre_means.set_index(["user","genre1"]))
test.reset_index(inplace=True)
test["user genre mean"].fillna(test["user mean"],inplace=True)
test.head(1)

In [ ]:
# weighted ensemble
stacking= linear_model.LinearRegression()

X = np.vstack([test['kmeans pred'].values,test['user median'].values])
X = np.vstack([X,test['user genre median'].values])
X = np.vstack([X,test['genre median'].values])
X = np.vstack([X,test['artist median'].values])
X = np.vstack([X,test['global median'].values])
X = np.vstack([X,test['user mean'].values])
X = np.vstack([X,test['user genre mean'].values])
X = np.vstack([X,test['genre mean'].values])
X = np.vstack([X,test['artist mean'].values])
X = X.T
X.shape

y = test['plays'].values

stacking.fit(X,y)
stacking.coef_

In [ ]:
test['ensemble'] =                            \
stacking.coef_[0]*test['kmeans pred']         \
+ stacking.coef_[1]*test['user median']       \
+ stacking.coef_[2]*test['user genre median'] \
+ stacking.coef_[3]*test['genre median']      \
+ stacking.coef_[4]*test['artist median']     \
+ stacking.coef_[5]*test['global median']     \
+ stacking.coef_[6]*test['user mean']
+ stacking.coef_[7]*test['user genre mean']   \
+ stacking.coef_[8]*test['genre mean']        \
+ stacking.coef_[9]*test['artist mean']      

test['AE'] = np.abs(test['plays']-test['ensemble'])  

In [ ]:
print "MAE is:", test.AE.mean()

In [ ]:
output_for_kaggle = test.copy()
output_for_kaggle.drop(["plays","closest center", "AE", "genre1","user","artist","kmeans pred","user genre median","user median", "genre median", "global median","artist median","artist mean","user mean", "genre mean", "user genre mean"],axis=1,inplace=True)
output_for_kaggle.rename(columns={"ensemble":"plays"},inplace = True)
output_for_kaggle.head()